# Pitchfork review analysis
### made by $DS^3$
June 26, 2019

In [7]:
# for database
import psycopg2

# for data science
import numpy as np
import pandas as pd
from functions_mod3 import import_data

# for statistics
from scipy import stats

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
genres_df = import_data("genres", 'genres_df')

In [14]:
reviews_df = import_data('reviews', 'reviews_df')


In [15]:
labels_df = import_data('labels', 'labels_df')

In [16]:
clean_reviews_df = reviews_df[['reviewid', 'score']]


In [17]:
reviews_labels_df = clean_reviews_df.merge(labels_df, how='left', on='reviewid')

In [18]:
music_df = reviews_labels_df.merge(genres_df, how='left', on='reviewid')

In [24]:
music_df.genre.unique()

array(['electronic', 'metal', 'rock', None, 'rap', 'experimental',
       'pop/r&b', 'folk/country', 'jazz', 'global'], dtype=object)

In [33]:
genre_reviews_df = music_df.pivot(columns='genre', values='score')

In [34]:
genre_reviews_df.head(1)

genre,nan,electronic,experimental,folk/country,global,jazz,metal,pop/r&b,rap,rock
0,NaN,9.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
music_df[music_df['genre'] == 'folk/country'].score.count()

735

In [49]:
country_scores = music_df[music_df['genre'] == 'folk/country'].score

In [50]:
electronic_scores = music_df[music_df['genre'] == 'electronic'].score

In [51]:
def welch_t(a, b):
    
    """ Calculate Welch's t statistic for two samples. """
    numerator = a.mean() - b.mean()
    
    # “ddof = Delta Degrees of Freedom”: the divisor used in the calculation is N - ddof, 
    #  where N represents the number of elements. By default ddof is zero.
    
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
    return np.abs(numerator/denominator)

In [56]:
t = welch_t(country_scores, electronic_scores)

In [54]:
music_df[music_df['genre'] == 'electronic'].score.size

4221

In [65]:
music_df.shape

(24909, 4)

In [ ]:
def welch_df(a, b):
    
    s1 = a.var(ddof=1)
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator

In [59]:
de_free = welch_df(country_scores, electronic_scores)

In [60]:
import scipy.stats as stats
p = 1 - stats.t.cdf(t, de_free)

In [61]:
p

1.6913470624047022e-11

The genre's were rated very differently between folk/country and electronic with a welch's P value of 1.69e-11

In [63]:
len(electronic_scores)

4221

In [64]:
len(country_scores)

735

In [62]:
music_df.head()

,reviewid,score,label,genre
0,22703,9.3,virgin,electronic
1,22721,7.9,hathenter,metal
2,22659,7.3,static shock,rock
3,22659,7.3,fashionable idiots,rock
4,22661,9.0,kill rock stars,rock


In [ ]:
music_df[music_df['genre'] == 'electronic'].score

In [68]:
touch_and_go_df = music_df[music_df['label'] =='touch and go']

In [69]:
touch_and_go_df.shape


(109, 4)

In [70]:
other_label_df = music_df[music_df['label'] !='touch and go']

In [71]:
other_label_df.shape

(24800, 4)

In [72]:
touch_and_go_scores = music_df[music_df['label'] =='touch and go'].score
other_scores = music_df[music_df['label'] !='touch and go'].score

In [73]:
#alpha = .05
t = welch_t(touch_and_go_scores, other_scores)
de_free = welch_df(touch_and_go_scores, other_scores)
p = 1 - stats.t.cdf(t, de_free)
p

0.263493455369455

The label doesn't matter for the ratings the p value = .26 for welch's t test
